# Evaluation в GigaLogger
В этом ноутбуке мы произведем оценку нашего RAG'а с помощью датасета и мощной LLM (gpt-4o)
И не только! Мы также замерим качество ответов на обычном GigaChat (без RAG) и замерим качество с обычным RAG.
У нас же в боте используется Adaptive RAG.

In [4]:
import os
os.environ["LANGFUSE_HOST"] = "https://gigalogger.demo.sberdevices.ru"
os.environ["LANGFUSE_PUBLIC_KEY"] = "..."
os.environ["LANGFUSE_SECRET_KEY"] = "..."

In [ ]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

In [6]:
from langfuse import Langfuse
langfuse = Langfuse()

## Цепочка для ответов

Определим промпт для цепочки, которая будет сравнивать наши ответы с правильными ответами

In [7]:
from langchain_core.prompts import PromptTemplate
COT_PROMPT = PromptTemplate(
    input_variables=["query", "context", "result"], template="""Ты учитель, оценивающий тест.
Тебе дан вопрос, контекст, к которому относится вопрос, и ответ студента. Тебе нужно оценить ответ студента как ПРАВИЛЬНЫЙ или НЕПРАВИЛЬНЫЙ, основываясь на контексте.
Опиши пошагово своё рассуждение, чтобы убедиться, что твой вывод правильный. Избегай просто указывать правильный ответ с самого начала.

Пример формата:
QUESTION: здесь вопрос
CONTEXT: здесь контекст, к которому относится вопрос
STUDENT ANSWER: здесь ответ студента
EXPLANATION: пошаговое рассуждение здесь
GRADE: CORRECT или INCORRECT здесь

Оценивай ответы студента ТОЛЬКО на основе их фактической точности. Игнорируй различия в пунктуации и формулировках между ответом студента и правильным ответом. Ответ студента может содержать больше информации, чем правильный ответ, если в нём нет противоречивых утверждений. Начнём!

QUESTION: "{query}"
CONTEXT: "{context}"
STUDENT ANSWER: "{result}"
EXPLANATION:"""
)

In [8]:
from langchain.evaluation import CotQAEvalChain
from langchain_openai import ChatOpenAI
def cot_llm(query, output, expected_output):
    eval_llm = ChatOpenAI(temperature=0)
    eval_chain = CotQAEvalChain.from_llm(llm=eval_llm, prompt=COT_PROMPT)
    resp = eval_chain.invoke({
        "query": query, "context": expected_output, "result": output
    })
    return eval_chain._prepare_output(resp)

Проверим работу цепочки оценки ответов

In [10]:
# Тут оценка неправильного ответа от LLM
cot_llm("Кто главный герой книги", "Кот", "Собака")

{'reasoning': 'Главный герой книги, контекст которой "Собака", скорее всего будет собакой, а не котом. Поэтому ответ студента "Кот" неверный.\nGRADE: INCORRECT',
 'value': 'INCORRECT',
 'score': 0}

In [11]:
# Тут оценка правильного ответа от LLM
cot_llm("Кто главный герой книги", "Кот", "Котик")

{'reasoning': 'Студент ответил "Кот", что является правильным ответом, так как главный герой книги действительно является котом. Нет противоречий в ответе студента, поэтому он оценивается как ПРАВИЛЬНЫЙ.\nGRADE: CORRECT',
 'value': 'CORRECT',
 'score': 1}

## Оценка
### Оценка ответов с обычным GigaChat

In [9]:
from langchain_community.chat_models import GigaChat
llm = GigaChat(model="GigaChat-Pro")

In [15]:
import tqdm
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_without_rag_2")
    try:
        generation = llm.invoke(input=item.input, config={"callbacks": [handler]}).content
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [06:50<00:00, 11.73s/it]


Первый прогон сделан. Смотрим результат...
![image.png](media/llm_without_rag_2.png)
Результат вышел `0.42`.
Судя по всему GigaChat хорошо справляется с вопросами сам о себе, но про GigaChain отвечает слабо.
Теперь попробуем прогнать датасет с простым RAG
### Оценка ответов GigaChat + RAG(стандартный)

In [11]:
from graph import vector_store
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_store.as_retriever())

In [12]:
qa_chain.invoke("Как обновить GigaChain?")

{'query': 'Как обновить GigaChain?',
 'result': 'Для обновления GigaChain выполните команду `bash pip install -U gigachain_community`.'}

In [14]:
import tqdm
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_with_rag_2")
    try:
        generation = qa_chain.invoke(input=item.input, config={"callbacks": [handler]})['result']
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [04:59<00:00,  8.56s/it]


Смотрим результат...
![image.png](media/llm_with_rag.png)
Результат вышел `0.68`.
### Оценка ответов GigaChat + Adaptive RAG

In [15]:
from graph import graph, GraphState

In [17]:
graph.invoke(input=GraphState(question="Как обновить GigaChain?"))['generation']

'Улучшенный ответ:\n\nЧтобы обновить GigaChain до последней версии, запустите следующую команду в терминале: `bash pip install -U gigachain_community`. Это позволит вам использовать все последние улучшения и исправления ошибок.'

In [18]:
import tqdm
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_with_arag")
    try:
        s = GraphState(question=item.input)
        generation = graph.invoke(input=s, config={"callbacks": [handler]})['generation']
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [09:29<00:00, 16.28s/it]


Смотрим результат...
![image.png](media/llm_with_arag.png)
Результат вышел `0.74`.